In [37]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [38]:
train = pd.read_csv('CS 583 Project/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('CS 583 Project/validation.csv')
test = pd.read_csv('CS 583 Project/test.csv')

In [39]:
train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train = train.loc[:12000, :]

In [40]:
xtrain, xvalid, ytrain, yvalid = train_test_split(
    train['comment_text'], train['toxic'], stratify=train['toxic'], random_state=42, test_size=0.2, shuffle=True
)

In [41]:
from transformers import AutoTokenizer, AutoModel, AdamW

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [42]:
class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=150):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        target = self.targets.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "target": torch.tensor(target, dtype=torch.float),
        }


In [43]:
train_dataset = ToxicCommentsDataset(xtrain, ytrain, tokenizer)
valid_dataset = ToxicCommentsDataset(xvalid, yvalid, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [44]:
class ToxicCommentClassifier(nn.Module):
    def __init__(self, pretrained_model_name, hidden_size=768):
        super(ToxicCommentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        output = self.dropout(pooled_output)
        return self.fc(output)


In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            targets = batch["target"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs.squeeze(-1), targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {total_loss/len(train_loader)}")

        # Validation
        model.eval()
        valid_preds = []
        valid_targets = []
        with torch.no_grad():
            for batch in valid_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                targets = batch["target"].to(device)

                outputs = model(input_ids, attention_mask)
                valid_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                valid_targets.extend(targets.cpu().numpy())

        # Convert predictions to binary values (0 or 1) based on a threshold (e.g., 0.5)
        valid_preds_binary = [1 if p >= 0.5 else 0 for p in valid_preds]

        # Calculate metrics
        auc = roc_auc_score(valid_targets, valid_preds)
        acc = accuracy_score(valid_targets, valid_preds_binary)
        precision = precision_score(valid_targets, valid_preds_binary)
        recall = recall_score(valid_targets, valid_preds_binary)
        f1 = f1_score(valid_targets, valid_preds_binary)
        cm = confusion_matrix(valid_targets, valid_preds_binary)

        print(f"Validation AUC: {auc}")
        print(f"Validation Accuracy: {acc}")
        print(f"Validation Precision: {precision}")
        print(f"Validation Recall: {recall}")
        print(f"Validation F1 Score: {f1}")
        print(f"Confusion Matrix:\n{cm}")


In [47]:
model = ToxicCommentClassifier("bert-base-uncased")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

# Train the model
train_model(model, train_loader, valid_loader, criterion, optimizer)


Epoch 1/3, Training Loss: 0.155218086897706
Validation AUC: 0.9814548387227506
Validation Accuracy: 0.965847563515202
Validation Precision: 0.8310502283105022
Validation Recall: 0.801762114537445
Validation F1 Score: 0.8161434977578476
Confusion Matrix:
[[2137   37]
 [  45  182]]
Epoch 2/3, Training Loss: 0.06449191097946216
Validation AUC: 0.981584525165249
Validation Accuracy: 0.9700124947938359
Validation Precision: 0.8475336322869955
Validation Recall: 0.8325991189427313
Validation F1 Score: 0.84
Confusion Matrix:
[[2140   34]
 [  38  189]]
Epoch 3/3, Training Loss: 0.025817780773892688
Validation AUC: 0.9765794390250822
Validation Accuracy: 0.9629321116201582
Validation Precision: 0.7974137931034483
Validation Recall: 0.8149779735682819
Validation F1 Score: 0.8061002178649237
Confusion Matrix:
[[2127   47]
 [  42  185]]
